In [1]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import requests 
import numpy as np


In [17]:
census = pd.read_csv('./DEC_00_SF1_H007.csv', header=1)

census.rename(columns={'Id2':'zip_code','Total:':'total',
                       'Not Hispanic or Latino householder:':'non_hispanic_tot',
                      'Hispanic or Latino householder:':'hispanic_tot'},inplace = True)
income = pd.read_csv('Income_Data.csv')
income.rename(columns={'Id2':'zip_code','Estimate; Per capita income in the past 12 months (in 2017 inflation-adjusted dollars)': 
                       'per_capita_income_2017_estimate',
                      'Margin of Error; Per capita income in the past 12 months (in 2017 inflation-adjusted dollars)':
                      'per_capita_income_2017_margin'},inplace = True)

In [26]:
#delete the puerto rican zip codes
# add back the trailing 0's for zip codes less than 5 digits
income['zip_code'] = income['zip_code'].astype('str')
fun = lambda x : '0'+ str(x) if len(x) == 4 else ('00'+ str(x) if len(x) == 3 else str(x))
income['zip_code'] = income['zip_code'].apply(fun)

In [27]:
print(type(income.loc[1,'zip_code']))
print(type(census.loc[1,'zip_code']))

<class 'str'>
<class 'str'>


In [28]:
#need to use inner join as there are some 991HH, 991XX zip codes in the census data with no income analysis
zip_income = pd.merge(census,income, on = 'zip_code',how = 'inner',suffixes =['_income','_zip'])
#delete the puerto rican zip codes
# add back the trailing 0's for zip codes less than 5 digits

#Moved the below section to an earlier line of code. Only the Income database has the type as int
#I think we may not all be on the same version of the .csv file. We should make sure this is cleaned up

#zip_income['zip_code'] = zip_income['zip_code'].astype('str')
#fun = lambda x : '0'+ str(x) if len(x) == 4 else ('00'+ str(x) if len(x) == 3 else str(x))
#zip_income['zip_code'] = zip_income['zip_code'].apply(fun)

In [29]:
#Calculate the total people for each race, regardless of hispanic or not
list_race = ['white_tot','black_tot','indian_tot','asian_tot','native_tot','other_tot','mixed_tot']
i = 1
for race in list_race:
    zip_income[race] = zip_income.iloc[:,3+i] + zip_income.iloc[:,11+i]
    i+= 1
    
zip_income.head()

,Id_income,zip_code,Geography_income,total,non_hispanic_tot,Not Hispanic or Latino householder: - Householder who is White alone,Not Hispanic or Latino householder: - Householder who is Black or African American alone,Not Hispanic or Latino householder: - Householder who is American Indian and Alaska Native alone,Not Hispanic or Latino householder: - Householder who is Asian alone,Not Hispanic or Latino householder: - Householder who is Native Hawaiian and Other Pacific Islander alone,...,Geography_zip,per_capita_income_2017_estimate,per_capita_income_2017_margin,white_tot,black_tot,indian_tot,asian_tot,native_tot,other_tot,mixed_tot
0,8600000US00601,00601,ZCTA5 00601,5895,27,23,2,0,1,0,...,ZCTA5 00601,7041.0,775.0,5895,5618,67,8,1,0,104
1,8600000US00602,00602,ZCTA5 00602,13520,98,79,6,0,3,0,...,ZCTA5 00602,8978.0,597.0,13520,11908,558,22,8,1,631
2,8600000US00603,00603,ZCTA5 00603,19152,229,185,19,1,16,0,...,ZCTA5 00603,10897.0,819.0,19152,15789,1028,37,33,4,1718
3,8600000US00606,00606,ZCTA5 00606,2013,13,11,1,0,1,0,...,ZCTA5 00606,5960.0,708.0,2013,1812,77,4,1,0,50
4,8600000US00610,00610,ZCTA5 00610,9262,73,61,6,0,5,0,...,ZCTA5 00610,9266.0,542.0,9262,7824,447,23,10,0,524


In [33]:
# subset the data and make sure that the zip code contains 5000 or more people

#BRIAN - Changed this to minimum 2500 people. I think there are small zip codes within larger metropolitan areas
census_sub = zip_income.loc[(zip_income['total'] >= 2500)&(~zip_income['zip_code'].str.startswith('00')),:]

In [45]:
zip_income.describe()

,total,non_hispanic_tot,Not Hispanic or Latino householder: - Householder who is White alone,Not Hispanic or Latino householder: - Householder who is Black or African American alone,Not Hispanic or Latino householder: - Householder who is American Indian and Alaska Native alone,Not Hispanic or Latino householder: - Householder who is Asian alone,Not Hispanic or Latino householder: - Householder who is Native Hawaiian and Other Pacific Islander alone,Not Hispanic or Latino householder: - Householder who is Some other race alone,Not Hispanic or Latino householder: - Householder who is Two or more races,hispanic_tot,...,Hispanic or Latino householder: - Householder who is Two or more races,per_capita_income_2017_estimate,per_capita_income_2017_margin,white_tot,black_tot,indian_tot,asian_tot,native_tot,other_tot,mixed_tot
count,31170.000000,31170.000000,31170.000000,31170.000000,31170.000000,31170.000000,31170.000000,31170.000000,31170.000000,31170.000000,...,31170.000000,31054.000000,31054.000000,31170.000000,31170.000000,31170.000000,31170.000000,31170.000000,31170.000000,31170.000000
mean,3417.410491,3082.229836,2532.925024,379.386205,21.179018,99.506352,3.025602,3.922169,42.285467,335.180654,...,18.928264,28819.493270,4718.154795,3417.410491,2714.649567,388.867661,24.621206,100.473051,3.342413,124.242862
std,4869.911638,4336.714278,3580.233867,1360.175612,88.940029,449.633561,30.948594,20.544314,104.096892,1372.518972,...,73.166442,12923.138588,6206.928458,4869.911638,3809.376668,1381.213444,92.475138,452.496935,32.177924,508.629758
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,218.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,319.000000,302.000000,265.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000,2.000000,...,0.000000,21444.250000,2063.000000,319.000000,282.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,1085.000000,1017.500000,882.500000,6.000000,3.000000,2.000000,0.000000,0.000000,7.000000,12.000000,...,1.000000,26263.500000,3256.000000,1085.000000,947.000000,6.000000,4.000000,2.000000,0.000000,4.000000
75%,4595.750000,4139.500000,3312.000000,146.000000,15.000000,25.000000,1.000000,2.000000,35.000000,102.000000,...,6.000000,32691.000000,5333.000000,4595.750000,3628.500000,154.000000,17.000000,26.000000,1.000000,34.000000
max,61774.000000,59353.000000,54283.000000,29244.000000,3649.000000,14115.000000,2668.000000,1485.000000,2953.000000,47691.000000,...,1662.000000,298129.000000,285548.000000,61774.000000,56150.000000,29326.000000,3667.000000,14142.000000,2819.000000,10713.000000


In [42]:
census_sub.describe()

,total,non_hispanic_tot,Not Hispanic or Latino householder: - Householder who is White alone,Not Hispanic or Latino householder: - Householder who is Black or African American alone,Not Hispanic or Latino householder: - Householder who is American Indian and Alaska Native alone,Not Hispanic or Latino householder: - Householder who is Asian alone,Not Hispanic or Latino householder: - Householder who is Native Hawaiian and Other Pacific Islander alone,Not Hispanic or Latino householder: - Householder who is Some other race alone,Not Hispanic or Latino householder: - Householder who is Two or more races,hispanic_tot,...,Hispanic or Latino householder: - Householder who is Two or more races,per_capita_income_2017_estimate,per_capita_income_2017_margin,white_tot,black_tot,indian_tot,asian_tot,native_tot,other_tot,mixed_tot
count,10688.000000,10688.000000,10688.000000,10688.000000,10688.000000,10688.000000,10688.000000,10688.000000,10688.000000,10688.000000,...,10688.000000,10688.000000,10688.000000,10688.000000,10688.000000,10688.000000,10688.000000,10688.000000,10688.000000,10688.000000
mean,8522.712294,7706.861714,6217.135105,1033.836265,41.921314,283.156718,8.036396,10.752058,112.023859,815.850580,...,48.239521,31759.235685,2291.131549,8522.712294,6630.235217,1051.298466,50.847399,285.706680,8.888567,335.472586
std,5204.960320,4644.741399,4010.921471,2170.403267,120.766930,732.950991,51.241732,33.936616,154.551802,1836.021477,...,105.416508,14134.626074,1463.395996,5204.960320,4154.028896,2201.360471,126.059893,737.444672,53.271927,820.862191
min,2500.000000,42.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,6934.000000,270.000000,2500.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4285.750000,3982.000000,3181.000000,45.000000,9.000000,17.000000,0.000000,1.000000,29.000000,58.000000,...,4.000000,22660.000000,1409.000000,4285.750000,3406.750000,47.000000,10.000000,18.000000,0.000000,17.000000
50%,7287.500000,6630.000000,5253.000000,248.000000,19.000000,66.500000,2.000000,4.000000,64.000000,183.000000,...,12.000000,28256.000000,1907.000000,7287.500000,5612.000000,257.000000,23.000000,68.000000,2.000000,62.000000
75%,11493.500000,10375.250000,8434.250000,1000.250000,38.000000,236.000000,4.000000,11.000000,135.000000,654.000000,...,41.000000,36991.500000,2671.000000,11493.500000,8946.000000,1024.000000,48.000000,238.000000,5.000000,248.000000
max,61774.000000,59353.000000,54283.000000,29244.000000,3649.000000,14115.000000,2668.000000,1485.000000,2953.000000,24489.000000,...,1505.000000,145479.000000,27484.000000,61774.000000,56150.000000,29326.000000,3667.000000,14142.000000,2819.000000,10713.000000


In [35]:
#BRIAN - Changed sample size to 250 for a more robust data set. Doing 5 samples to compare.
# randomly select 100 zip code out of subset census data
census_sub_sample1 = census_sub.sample(n = 250,random_state=100).reset_index(drop = True)
census_sub_sample2 = census_sub.sample(n = 250,random_state=99).reset_index(drop = True)
census_sub_sample3 = census_sub.sample(n = 250,random_state=98).reset_index(drop = True)
census_sub_sample4 = census_sub.sample(n = 250,random_state=97).reset_index(drop = True)
census_sub_sample5 = census_sub.sample(n = 250,random_state=96).reset_index(drop = True)


In [38]:
#census_sub_sample1.head(10)

In [39]:
census_sub_sample1A = census_sub_sample1.drop(columns=['Geography_income','Id_income','Geography_zip'])
census_sub_sample2A = census_sub_sample2.drop(columns=['Geography_income','Id_income','Geography_zip'])
census_sub_sample3A = census_sub_sample3.drop(columns=['Geography_income','Id_income','Geography_zip'])
census_sub_sample4A = census_sub_sample4.drop(columns=['Geography_income','Id_income','Geography_zip'])
census_sub_sample5A = census_sub_sample5.drop(columns=['Geography_income','Id_income','Geography_zip'])
#census_sub_sample1A.head(10)

In [77]:
#census_sub_sample_short.to_csv('sample_census2_100.csv')

In [47]:
census_sub_sample1A.to_csv('250_sample_1.csv', index=False)
census_sub_sample2A.to_csv('250_sample_2.csv', index=False)
census_sub_sample3A.to_csv('250_sample_3.csv', index=False)
census_sub_sample4A.to_csv('250_sample_4.csv', index=False)
census_sub_sample5A.to_csv('250_sample_5.csv', index=False)